In [19]:
%%bash

(zgrep ^"#" /research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.gz; zgrep -v ^"#" /research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.gz | sort -k1,1 -k4,4n) | bgzip > /research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.bgz;
tabix -p gff /research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.bgz

In [22]:
import pysam

In [23]:
tb= pysam.TabixFile("/research/references/hsapiens/hg38/hg38.ncbiRefSeq.gtf.bgz")

In [70]:
def get_features(chromosome, position, gff_tabix):
    """
    chromosome [STR] that is the contig/sequence/chromosome the mutation is on
    position [INT] the 1-based position of the mutation
    gff_tabix [PYSAM TABIXFILE OBJECT] this is a tabix indexed file that has been opened in pysam with  pysam.TabixFile(filename)

    Returns list of all features that overlap the mutation
    """
    feature_types = list(set([l.split()[2] for l in gff_tabix.fetch(chromosome, position,position+1)]))
    if len(feature_types) == 0:
        return ["intergenic"]
    else:
        return feature_types

    
    
    
class AnnotationError(Exception):
       pass


def annotate_site(chromosome, position, gff_tabix):
    """
    Uses some funky logic to identify 
    """
    features = get_features(chromosome, position, gff_tabix)
    if "intergenic" in features:
        return "intergenic"
    elif "CDS" in features:
        return "coding"
    elif "3UTR" in features or "5UTR" in features:
        return "utr"
    elif "exon" in features:
        # I don't really know what these are 
        return "non_coding_nonUTR_exonic"
    elif "transcript" in features:
        # this is probably introns but I am not writing introns because I wonder if I am forgetting something else
        return "intronic"
    else:
        # if it still isn't annotated - stop what you're doing and look at the feature list
        # probably modify the code
        raise AnnotationError("FeatureList is weird AF:\t" + str(features))     

# Annotating Random sites 

In [74]:
import random    


N=0
while N < 100000:
    N+=1
    chromosome = "chr" + str( random.randint(1,22) )
    position  = random.randint(100000,5000000)
#     print(chromosome,position, sep="\t", end="\t")
#     print(annotate_site(chromosome, position, tb), end= "\t")
#     print(get_features(chromosome, position, tb))
    a=annotate_site(chromosome, position, tb)


In [65]:
get_features("chr16", 4845075,tb)

['5UTR', 'exon', 'transcript', 'CDS']